In [1]:
!pip install --upgrade pip
!pip install gcsfs

Requirement already up-to-date: pip in /opt/conda/anaconda/lib/python3.6/site-packages (20.0.2)


In [2]:
%%time
import pandas as pd
import os, sys, time, json, re, string, datetime

from pyspark import SparkContext, SparkConf, StorageLevel, keyword_only
from pyspark.sql.types import IntegerType

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.ml.param.shared import HasInputCol, HasInputCols, HasOutputCol, HasOutputCols, Param
from pyspark.ml.feature import OneHotEncoder, HashingTF, IDF, Tokenizer, RegexTokenizer, NGram, CountVectorizer
from pyspark.ml.feature import StopWordsRemover, VectorAssembler, PCA, OneHotEncoderEstimator,StringIndexer

from pyspark.ml.classification import LogisticRegression, NaiveBayes, DecisionTreeClassifier, RandomForestClassifier
from pyspark.ml.classification import GBTClassifier, MultilayerPerceptronClassifier

from pyspark.ml import Pipeline, Transformer

from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from platform import python_version
print(python_version())

3.6.10
CPU times: user 52.1 ms, sys: 15.7 ms, total: 67.7 ms
Wall time: 70.5 ms


In [3]:
%%time
sc

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 7.15 µs


<SparkContext master=yarn appName=PySparkShell>

In [4]:
%%time
spark = SparkSession.builder \
        .appName("fakenews") \
        .config("spark.master", "yarn") \
        .config("spark.submit.deployMode", "cluster") \
        .config("spark.driver.memory", "25g") \
        .config("spark.executor.instances", "5") \
        .config("spark.executor.cores", "4") \
        .config("spark.executor.memory", "25g") \
        .getOrCreate()

CPU times: user 3.95 ms, sys: 4.07 ms, total: 8.01 ms
Wall time: 9.67 ms


In [5]:
%%time
df_fake_train = pd.read_csv('gs://weicheng30417/data/fake-news/100k_fake_news_cleaned_dataset.csv')
df_fake_train[['authors_missing']] = df_fake_train[['authors_missing']].astype(int)
df_fake_train['label'] = 1
df_reliable_train = pd.read_csv('gs://weicheng30417/data/fake-news/100k_reliable_news_cleaned_dataset.csv')
df_reliable_train[['authors_missing']] = df_reliable_train[['authors_missing']].astype(int)
df_reliable_train['label'] = 0
df_news_train = pd.concat([df_fake_train, df_reliable_train])
df_news_train[df_news_train.columns] = df_news_train[df_news_train.columns].astype(str)
print(df_news_train.info())
df_news_train

<class 'pandas.core.frame.DataFrame'>
Int64Index: 217806 entries, 0 to 108010
Data columns (total 7 columns):
domain             217806 non-null object
type               217806 non-null object
content            217806 non-null object
title              217806 non-null object
authors            217806 non-null object
authors_missing    217806 non-null object
label              217806 non-null object
dtypes: object(7)
memory usage: 13.3+ MB
None
CPU times: user 9.77 s, sys: 1.43 s, total: 11.2 s
Wall time: 29.9 s


,domain,type,content,title,authors,authors_missing,label
0,beforeitsnews.com,fake,Boehner presses to make tax cuts permanent\n\n...,Boehner presses to make tax cuts permanent,United Liberty,0,1
1,beforeitsnews.com,fake,An Armed Good Samaritan Who Doesn’t Want The S...,An Armed Good Samaritan Who Doesn’t Want The S...,The Real Revo,0,1
2,beforeitsnews.com,fake,(Before It's News)\n\nby Rob Morphy\n\nLegends...,Village Of The Dead: The Anjikuni Mystery,"Rob Morphy, Mort Amsel",0,1
3,teaparty.org,fake,(Breitbart) – With his hysterical gotcha attac...,Trump Calls Out Race-Baiting ABC News Reporter,nan,1,1
4,beforeitsnews.com,fake,Researchers develop new depression diagnosis a...,Researchers develop new depression diagnosis a...,Bel Marra Health,0,1
...,...,...,...,...,...,...,...
108006,sports.yahoo.com,reliable,"View photos\nMichigan guard Zak Irvin, right, ...",No. 25 Michigan beats Mount St. Mary's 64-47,The Associated Press,0,0
108007,uk.finance.yahoo.com,reliable,President-elect Donald Trump continued his scr...,"Trump quotes Hillary Clinton, rages against Wi...",Maxwell Tani,0,0
108008,www.yahoo.com,reliable,Holiday shoppers eager to snag big discounts t...,Dialing up deals: Black Friday online sales hi...,ALEX VEIGA,0,0
108009,www.reuters.com,reliable,Kabul police raid shisha cafes in crackdown on...,Kabul police raid shisha cafes in crackdown on...,nan,1,0


In [6]:
%%time
df_news = spark.createDataFrame(df_news_train)
df_news = df_news.withColumn("label", df_news["label"].cast(IntegerType()))
df_news = df_news.withColumn("authors_missing", df_news["authors_missing"].cast(IntegerType()))
df_news.show(10)

+-----------------+----+--------------------+--------------------+--------------------+---------------+-----+
|           domain|type|             content|               title|             authors|authors_missing|label|
+-----------------+----+--------------------+--------------------+--------------------+---------------+-----+
|beforeitsnews.com|fake|Boehner presses t...|Boehner presses t...|      United Liberty|              0|    1|
|beforeitsnews.com|fake|An Armed Good Sam...|An Armed Good Sam...|       The Real Revo|              0|    1|
|beforeitsnews.com|fake|(Before It's News...|Village Of The De...|Rob Morphy, Mort ...|              0|    1|
|     teaparty.org|fake|(Breitbart) – Wit...|Trump Calls Out R...|                 nan|              1|    1|
|beforeitsnews.com|fake|Researchers devel...|Researchers devel...|    Bel Marra Health|              0|    1|
|beforeitsnews.com|fake|Trading Watch Lis...|Trading Watch Lis...|Bulls On Wall Street|              0|    1|
|beforeits

In [7]:
%%time
#remove empty content which will cause problem when transform the text
df_news = df_news.filter(df_news.content != "")
df_news = df_news.filter(df_news.title != "")
df_news = df_news.dropDuplicates(['label', 'content', 'title', 'authors_missing'])

CPU times: user 7.67 ms, sys: 0 ns, total: 7.67 ms
Wall time: 615 ms


In [8]:
# only keep type and content
df_news = df_news.select('label', 'content', 'title','authors_missing')

# split the dataset
df_train, df_test = df_news.randomSplit([0.8, 0.2], seed=666)
param_tuning = False

In [9]:
%%time
# customized transformer class to manually extract some counting based text features
class ReviewContentTransformer(Transformer, HasInputCol, HasOutputCol):

    @keyword_only
    def __init__(self, inputCol="content", outputCol="content_features"):
        super(ReviewContentTransformer, self).__init__()
        kwargs = self._input_kwargs
        self.setParams(**kwargs)

    @keyword_only
    def setParams(self, inputCol=None, outputCol=None):
        kwargs = self._input_kwargs
        return self._set(**kwargs)


    def _transform(self, dataset):
        
        def f(s):
            uppercase_count = 0
            char_count = 0
            for c in s:                
                if c in string.ascii_uppercase:
                    uppercase_count += 1
                    char_count += 1
                elif c in string.ascii_lowercase:
                    char_count += 1
            
            text_len = len(s)
            return Vectors.dense(text_len, char_count, 
                                 uppercase_count, uppercase_count / (char_count + 1e-10))

        return dataset.withColumn(self.getOutputCol(), 
                                  F.udf(f, VectorUDT())(dataset[self.getInputCol()]))

CPU times: user 113 µs, sys: 12 µs, total: 125 µs
Wall time: 129 µs


In [10]:
%%time
# customized transformer class to manually extract some counting based word features
class ReviewWordsTransformer(Transformer, HasInputCol, HasOutputCol):

    @keyword_only
    def __init__(self, inputCol="content", outputCol="content_features"):
        super(ReviewWordsTransformer, self).__init__()
        kwargs = self._input_kwargs
        self.setParams(**kwargs)

    @keyword_only
    def setParams(self, inputCol=None, outputCol=None):
        kwargs = self._input_kwargs
        return self._set(**kwargs)


    def _transform(self, dataset):
        
        def f(words):    
            word_count = len(words)
            unique_word_count = len(set(words))
            upper_words = []
            for w in words:
                if w.isupper():
                    upper_words.append(w)
            upper_word_count = len(set(upper_words))
            unique_upper_word_count = len(upper_words)
            return Vectors.dense(word_count, unique_word_count, unique_word_count / (word_count + 1e-10),
                                 upper_word_count, upper_word_count / (word_count + 1e-10), 
                                 unique_upper_word_count, unique_upper_word_count / (upper_word_count + 1e-10))

        return dataset.withColumn(self.getOutputCol(), 
                                  F.udf(f, VectorUDT())(dataset[self.getInputCol()]))

CPU times: user 120 µs, sys: 12 µs, total: 132 µs
Wall time: 138 µs


In [11]:
%%time
# show model prediction performance on the given dataset
def eval_model_perf(fitted_model, dataset, label_col="label", prediction_col="prediction", probability_col="probability"):
    pred_dataset = fitted_model.transform(dataset)
    eval_dataset = pred_dataset.select(label_col, prediction_col, probability_col)
    # model performance evaluation
    metricNames = ["accuracy", "f1"]
    model_eval = MulticlassClassificationEvaluator(predictionCol=prediction_col, labelCol=label_col)
    for m in metricNames:
        val = model_eval.evaluate(eval_dataset, {model_eval.metricName: m})
        print(m, " = ", val)
    roc_eval = BinaryClassificationEvaluator(rawPredictionCol=probability_col, labelCol=label_col, metricName="areaUnderROC")
    print("AUC =", roc_eval.evaluate(eval_dataset))    
    return pred_dataset

# show CV param tunning result
def show_cv_results(cv_model):
    for result, param in sorted(zip(cv_model.avgMetrics, cv_model.getEstimatorParamMaps()), reverse=True, key=lambda x: x[0]):
        print(result, " | ", param)

CPU times: user 9 µs, sys: 0 ns, total: 9 µs
Wall time: 16 µs


In [12]:
%%time
def build_data_preproc_model_with_pca(vocab_size=5000):
    preproc_steps = [
        RegexTokenizer(inputCol="content", outputCol="all_words", pattern=r"\W"),
        StopWordsRemover(inputCol="all_words", outputCol="words"),
        CountVectorizer(inputCol="words", outputCol="tf_features", vocabSize=vocab_size),
        IDF(inputCol="tf_features", outputCol="tfidf_features"),
        PCA(inputCol="tfidf_features", outputCol="pca_features", k=100),
        
        ReviewContentTransformer(inputCol="content", outputCol="content_features"),
        ReviewWordsTransformer(inputCol="words", outputCol="word_features"),
        
        RegexTokenizer(inputCol="title", outputCol="all_title_words", pattern=r"\W"),
        StopWordsRemover(inputCol="all_title_words", outputCol="title_words"),
        CountVectorizer(inputCol="title_words", outputCol="title_tf_features", vocabSize=100),
        IDF(inputCol="title_tf_features", outputCol="title_tfidf_features"),
        PCA(inputCol="title_tfidf_features", outputCol="title_pca_features", k=100),   
        
        StringIndexer(inputCol="authors_missing", outputCol="authors_missing_indexed", handleInvalid='keep'),
        OneHotEncoder(inputCol="authors_missing_indexed", outputCol="authors_missing_feature"),
        
        VectorAssembler(inputCols=["pca_features", "title_pca_features", 
                                   "content_features", "word_features", "authors_missing_feature"], 
                        outputCol="features")
    ]
    return Pipeline(stages=preproc_steps)

def build_data_preproc_model_without_pca(vocab_size=5000):
    preproc_steps = [
        RegexTokenizer(inputCol="content", outputCol="all_words", pattern=r"\W"),
        StopWordsRemover(inputCol="all_words", outputCol="words"),
        CountVectorizer(inputCol="words", outputCol="tf_features", vocabSize=vocab_size),
        IDF(inputCol="tf_features", outputCol="tfidf_features"),
        
        RegexTokenizer(inputCol="title", outputCol="all_title_words", pattern=r"\W"),
        StopWordsRemover(inputCol="all_title_words", outputCol="title_words"),
        CountVectorizer(inputCol="title_words", outputCol="title_tf_features", vocabSize=100),
        IDF(inputCol="title_tf_features", outputCol="title_tfidf_features"),
        
        StringIndexer(inputCol="authors_missing", outputCol="authors_missing_indexed", handleInvalid='keep'),
        OneHotEncoder(inputCol="authors_missing_indexed", outputCol="authors_missing_feature"),
        
        ReviewContentTransformer(inputCol="content", outputCol="content_features"),
        ReviewWordsTransformer(inputCol="words", outputCol="word_features"),
        
        VectorAssembler(inputCols=["tfidf_features", "title_tfidf_features", 
                                   "content_features", "word_features", "authors_missing_feature"],
                        outputCol="features")
    ]
    return Pipeline(stages=preproc_steps)

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 11.4 µs


In [13]:
%%time
print("**********Run Models with PCA Features**********")
# generate the features to be used for model training
preproc_model = build_data_preproc_model_with_pca(2000).fit(df_train)
df_train_pca = preproc_model.transform(df_train).select("label", "features")
df_test_pca = preproc_model.transform(df_test).select("label", "features")

**********Run Models with PCA Features**********
CPU times: user 686 ms, sys: 236 ms, total: 922 ms
Wall time: 2min 42s


In [14]:
%%time
lr_model = LogisticRegression(featuresCol='features', 
                              labelCol='label', 
                              predictionCol='prediction', 
                              probabilityCol='probability', 
                              rawPredictionCol='rawPrediction',
                              family='binomial', 
                              fitIntercept=True, 
                              threshold=0.5, 
                              standardization=False, 
                              maxIter=200, 
                              regParam=0.005, 
                              elasticNetParam=0, 
                              tol=1e-06, 
                              aggregationDepth=2)

lr_model = lr_model.fit(df_train_pca)
eval_model_perf(lr_model, df_test_pca)  

accuracy  =  0.8898226819357222
f1  =  0.8897138135448337
AUC = 0.9528308940343868
CPU times: user 4 s, sys: 1.95 s, total: 5.95 s
Wall time: 3min 40s


DataFrame[label: int, features: vector, rawPrediction: vector, probability: vector, prediction: double]

In [15]:
%%time
dt_model = DecisionTreeClassifier(featuresCol='features', 
                                  labelCol='label', 
                                  predictionCol='prediction', 
                                  probabilityCol='probability', 
                                  rawPredictionCol='rawPrediction', 
                                  maxDepth=10, maxBins=32, 
                                  minInstancesPerNode=1, 
                                  minInfoGain=0.0, 
                                  maxMemoryInMB=2048, 
                                  cacheNodeIds=True, 
                                  checkpointInterval=10,
                                  impurity='gini', 
                                  seed=666)

dt_model = dt_model.fit(df_train_pca)
eval_model_perf(dt_model, df_test_pca)

accuracy  =  0.8279691540450683
f1  =  0.8279438999270583
AUC = 0.8989649256198621
CPU times: user 655 ms, sys: 237 ms, total: 892 ms
Wall time: 5min 14s


DataFrame[label: int, features: vector, rawPrediction: vector, probability: vector, prediction: double]

In [ ]:
%%time
rf_model = RandomForestClassifier(featuresCol='features', 
                                  labelCol='label', 
                                  predictionCol='prediction', 
                                  probabilityCol='probability', 
                                  rawPredictionCol='rawPrediction',
                                  maxDepth=10, 
                                  maxBins=32, 
                                  minInstancesPerNode=1, 
                                  minInfoGain=0.0, 
                                  maxMemoryInMB=2048, 
                                  cacheNodeIds=True, 
                                  checkpointInterval=10, 
                                  impurity='gini', 
                                  numTrees=200, 
                                  featureSubsetStrategy='auto', 
                                  seed=666, 
                                  subsamplingRate=0.8)

rf_model = rf_model.fit(df_train_pca)
eval_model_perf(rf_model, df_test_pca)

accuracy  =  0.868235131141485
f1  =  0.8682193093793285
AUC = 0.9429120190508549
CPU times: user 755 ms, sys: 254 ms, total: 1.01 s
Wall time: 8min 2s


DataFrame[label: int, features: vector, rawPrediction: vector, probability: vector, prediction: double]

In [ ]:
%%time
gbt_model = GBTClassifier(featuresCol='features', 
                         labelCol='label', 
                         maxIter=250)

gbt_model = gbt_model.fit(df_train_pca)
eval_model_perf(gbt_model, df_test_pca)

accuracy  =  0.9045760990025858
f1  =  0.9045747967490159
AUC = 0.9684544232585017
CPU times: user 40.4 s, sys: 17.1 s, total: 57.5 s
Wall time: 40min 9s


DataFrame[label: int, features: vector, rawPrediction: vector, probability: vector, prediction: double]

In [ ]:
%%time
print("**********Run Models without PCA Features**********")
# generate the features to be used for model training
preproc_model = build_data_preproc_model_without_pca(3000).fit(df_train)
df_train_wo_pca = preproc_model.transform(df_train).select("label", "features")
df_test_wo_pca = preproc_model.transform(df_test).select("label", "features")

**********Run Models without PCA Features**********
CPU times: user 482 ms, sys: 98.4 ms, total: 580 ms
Wall time: 1min 10s


In [ ]:
%%time
lr_model = LogisticRegression(featuresCol='features', 
                              labelCol='label', 
                              predictionCol='prediction', 
                              probabilityCol='probability', 
                              rawPredictionCol='rawPrediction',
                              family='binomial', 
                              fitIntercept=True, 
                              threshold=0.5, 
                              standardization=False, 
                              maxIter=200, 
                              regParam=0.005, 
                              elasticNetParam=0, 
                              tol=1e-06, 
                              aggregationDepth=2)

lr_model = lr_model.fit(df_train_wo_pca)    
eval_model_perf(lr_model, df_test_wo_pca)

accuracy  =  0.9532000369412634
f1  =  0.9531987326791072
AUC = 0.9884873734361753
CPU times: user 3.96 s, sys: 2.07 s, total: 6.02 s
Wall time: 4min 12s


DataFrame[label: int, features: vector, rawPrediction: vector, probability: vector, prediction: double]

In [ ]:
%%time
dt_model = DecisionTreeClassifier(featuresCol='features', 
                                  labelCol='label', 
                                  predictionCol='prediction', 
                                  probabilityCol='probability', 
                                  rawPredictionCol='rawPrediction', 
                                  maxDepth=10, maxBins=32, 
                                  minInstancesPerNode=1, 
                                  minInfoGain=0.0, 
                                  maxMemoryInMB=2048, 
                                  cacheNodeIds=True, 
                                  checkpointInterval=10,
                                  impurity='gini', 
                                  seed=666)

dt_model = dt_model.fit(df_train_wo_pca)
eval_model_perf(dt_model, df_test_wo_pca)

accuracy  =  0.9307582194311046
f1  =  0.9307362530544104
AUC = 0.972280896261124
CPU times: user 564 ms, sys: 239 ms, total: 804 ms
Wall time: 3min 57s


DataFrame[label: int, features: vector, rawPrediction: vector, probability: vector, prediction: double]

In [ ]:
%%time
rf_model = RandomForestClassifier(featuresCol='features', 
                                  labelCol='label', 
                                  predictionCol='prediction', 
                                  probabilityCol='probability', 
                                  rawPredictionCol='rawPrediction',
                                  maxDepth=10, 
                                  maxBins=32, 
                                  minInstancesPerNode=1, 
                                  minInfoGain=0.0, 
                                  maxMemoryInMB=2048, 
                                  cacheNodeIds=True, 
                                  checkpointInterval=10, 
                                  impurity='gini', 
                                  numTrees=200, 
                                  featureSubsetStrategy='auto', 
                                  seed=666, 
                                  subsamplingRate=0.8)

rf_model = rf_model.fit(df_train_wo_pca)
eval_model_perf(rf_model, df_test_wo_pca)

accuracy  =  0.9313354266715922
f1  =  0.9313173358398661
AUC = 0.9846590644542499
CPU times: user 733 ms, sys: 189 ms, total: 921 ms
Wall time: 8min 17s


DataFrame[label: int, features: vector, rawPrediction: vector, probability: vector, prediction: double]

In [ ]:
%%time
gbt_model = GBTClassifier(featuresCol='features', 
                         labelCol='label', 
                         maxIter=250)

gbt_model = gbt_model.fit(df_train_wo_pca)
eval_model_perf(gbt_model, df_test_wo_pca)

accuracy  =  0.9589951976357591
f1  =  0.9589939700573036
AUC = 0.9944275714534736
CPU times: user 46 s, sys: 18.9 s, total: 1min 4s
Wall time: 1h 39min 53s


DataFrame[label: int, features: vector, rawPrediction: vector, probability: vector, prediction: double]

In [ ]:
%%time
nb_model = NaiveBayes(featuresCol='features', 
                              labelCol='label', 
                              predictionCol='prediction', 
                              probabilityCol='probability', 
                              rawPredictionCol='rawPrediction', 
                              smoothing=1, 
                              modelType='multinomial')

nb_model = nb_model.fit(df_train_wo_pca)
eval_model_perf(nb_model, df_test_wo_pca)

accuracy  =  0.8692510158847433
f1  =  0.8692045661241024
AUC = 0.9086540366430549
CPU times: user 274 ms, sys: 96.9 ms, total: 371 ms
Wall time: 2min


DataFrame[label: int, features: vector, rawPrediction: vector, probability: vector, prediction: double]